In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nb
import pandas as pd
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [3]:
annot = np.array(nb.load('/home/julia/data/gradients/atlas/allen_api/annotation.nii.gz').get_data(), dtype='float64')
gradient = nb.load('/home/julia/data/gradients/results/gradient0000_highres_allen.nii.gz').get_data()
mask = nb.load('/home/julia/data/gradients/atlas/cortex/cortex_mask_25um_allen.nii.gz').get_data()

In [4]:
mcc = MouseConnectivityCache(manifest_file=
                             '/home/julia/data/gradients/atlas/allen_api/mouse_connectivity_manifest.json')
structure_tree = mcc.get_structure_tree()
structure_map = structure_tree.get_name_map()

In [5]:
gradient[mask==0] = np.nan
annot[mask==0] = np.nan

In [6]:
unique_annot = np.unique(annot)
unique_annot = unique_annot[np.isnan(unique_annot)==0]
unique_annot = unique_annot[1:]

In [7]:
region_means = []
region_names=[]
for region in range(unique_annot.shape[0]):
    region_means.append(np.mean(gradient[annot==unique_annot[region]]))
    region_names.append(structure_map[unique_annot[region]])

In [12]:
df = pd.DataFrame(data=zip(unique_annot, region_names, region_means), 
                  columns=['idx', 'name', 'mean'])

In [20]:
df = df.sort_values(by=['mean'])

In [22]:
df.to_csv('/home/julia/data/gradients/results/regions_by_gradient/region_means_gradient0000.csv')

### Try broader region definitions

In [8]:
parents = []
unique_parents = set()
for child in unique_annot:
    parents.append(structure_tree.parent_ids([child])[0])
    unique_parents.add(structure_tree.parent_ids([child])[0])
unique_parents.remove(None)
unique_parents = list(unique_parents)

In [196]:
parents=np.array(parents)
region_means = np.array(region_means)

In [203]:
parent_means = []
parent_names = []
for parent in unique_parents:
    parent_means.append(np.mean(region_means[parents==parent]))
    parent_names.append(structure_map[parent])

In [204]:
df_broad = pd.DataFrame(data=zip(unique_parents, parent_names, parent_means), 
                        columns=['idx', 'name', 'mean'])

In [206]:
df_broad = df_broad.sort_values(by=['mean'])

In [208]:
df_broad.to_csv('/home/julia/data/gradients/results/regions_by_gradient/parent_means_gradient0000.csv')

### Curate parent list through grandparents

In [15]:
grandparents = set()
for parent in unique_parents:
    grandparents.add(structure_map[structure_tree.parent_ids([parent])[0]])

In [16]:
grandparents

{'Agranular insular area',
 'Anterior cingulate area',
 'Auditory areas',
 'Cerebral cortex',
 'Cerebrum',
 'Cortical plate',
 'Cortical subplate',
 'Entorhinal area',
 'Hippocampal formation',
 'Isocortex',
 'Midbrain, sensory related',
 'Olfactory areas',
 'Orbital area',
 'Posterior parietal association areas',
 'Primary somatosensory area',
 'Retrosplenial area',
 'Somatomotor areas',
 'Somatosensory areas',
 'Striatum',
 'Visual areas',
 'cerebrum related',
 'corpus callosum',
 'fiber tracts',
 'lateral forebrain bundle system',
 'medial forebrain bundle system',
 'root'}

In [37]:
curated_grandparents = grandparents.copy()
curated_grandparents.remove('Cerebral cortex')
curated_grandparents.remove('Cerebrum')
curated_grandparents.remove('Cortical plate')
curated_grandparents.remove('Cortical subplate')
curated_grandparents.remove('Isocortex')
curated_grandparents.remove('Striatum')
curated_grandparents.remove('cerebrum related')
curated_grandparents.remove('corpus callosum')
curated_grandparents.remove('fiber tracts')
curated_grandparents.remove('lateral forebrain bundle system')
curated_grandparents.remove('medial forebrain bundle system')
curated_grandparents.remove('root')
curated_grandparents.remove('Midbrain, sensory related')

In [42]:
curated_parents = []
for parent in unique_parents:
    if structure_map[structure_tree.parent_ids([parent])[0]] in curated_grandparents:
        curated_parents.append(parent)

In [58]:
parents=np.array(parents)
region_means = np.array(region_means)

In [64]:
curated_parent_means = []
curated_parent_names = []
for parent in curated_parents:
    curated_parent_means.append(np.mean(region_means[parents==parent]))
    curated_parent_names.append(structure_map[parent])

In [65]:
df_curated = pd.DataFrame(data=zip(curated_parents, curated_parent_names, curated_parent_means), 
                          columns=['idx', 'name', 'mean'])

In [66]:
df_curated = df_curated.sort_values(by=['mean'])

In [67]:
df_curated.to_csv('/home/julia/data/gradients/results/regions_by_gradient/curated_means_gradient0000.csv')

In [70]:
np.save('/home/julia/data/gradients/results/regions_by_gradient/curated_parents.npy', curated_parents)